In [1]:
from pyspark import SparkContext
# import matplotlib.pyplot as plt

sc.stop()
sc = SparkContext(appName="exercise")
file=sc.textFile("temperature-readings-tiny.csv")
linesp = file.map(lambda line:line.split(";"))

stations_Ostergotland=sc.textFile("stations.csv")
liness = stations_Ostergotland.map(lambda line:line.split(";"))

######## the list of all the stations
stations = liness.map(lambda x:x[0]).distinct().collect()  

################ step 1 ###############
daily_temp = linesp.map(lambda x:((x[1][0:7],x[0]),float(x[3])))
daily_temp = daily_temp.filter(lambda x: str(x[0][1]) in stations)
month_temp_sum = daily_temp.reduceByKey(lambda v1,v2:v1+v2)    
# print(*month_temp_sum.take(10),sep="\n")

num_month = daily_temp.map(lambda x:(x[0], 1))    
num_month = num_month.reduceByKey(lambda v1,v2:v1+v2)       
# print(*num_month.take(10), sep="\n")
month_temp_ave = month_temp_sum.union(num_month).reduceByKey(lambda v1,v2:v1/v2)
# print(*month_temp_ave.take(10),sep="\n")

################ step 2 ###############
month_temp_ave2 = month_temp_ave.map(lambda x: (x[0][0], x[1]))
month_temp_ave2 = month_temp_ave2.reduceByKey(lambda v1,v2: v1+v2)
num_station = month_temp_ave.map(lambda x: (x[0][0], 1))
num_station = num_station.reduceByKey(lambda v1,v2:v1+v2)
month_temp_ave2 = month_temp_ave2.union(num_station).reduceByKey(lambda v1,v2:v1/v2)
month_temp_ave2 = month_temp_ave2.map(lambda x: [int(x[0][0:4]), int(x[0][5:7]), x[1]])
# print(*month_temp_ave2.take(10),sep="\n")

############### final ##################
# list[year, month, value]
ave1 = month_temp_ave2.filter(lambda x: x[0]>=1950 and x[0]<=2014)\
					  .sortBy(lambda x: (x[0],[1]), ascending = False)
final = ave1.collect()
# dict(month: value)
ave2 = month_temp_ave2.filter(lambda x: x[0]>=1950 and x[0]<=1980)\
					  .sortBy(lambda x: (x[0],[1]), ascending = False)
sum2 = ave2.map(lambda x: (x[1], x[2])).reduceByKey(lambda v1, v2: v1+v2)
num2 = ave2.map(lambda x: (x[1], 1)).reduceByKey(lambda v1, v2: v1+v2)
ave2 = dict(sum2.union(num2).reduceByKey(lambda v1,v2:v1/v2).collect())

for i in range(len(final)):
    month = final[i][1]
    final[i][2] -= ave2[month]
    
lll = sc.parallelize(final)
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(lll)
df = df.sort(df._1.desc(), df._2.desc()).withColumnRenamed("_1","year")\
							 .withColumnRenamed("_2","month")\
							 .withColumnRenamed("_3","difference")
df.show()
# fff = df.rdd
# fff.saveAsTextFile("./results1/1q6") 

#with open("1q6", "w") as file:
#    for element in final:
#        file.write("%s, %s, %s \n" % (element[0], element[1], element[2]))
        
#a = ave1.map(lambda x: (str(x[0])+"-"+str(x[1]), x[2]))
#kk = a.keys().collect()
#fig, ax = plt.subplots(figsize=(8,4))
#ax.plot(kk, [x[2] for x in final])


+----+-----+-------------------+
|year|month|         difference|
+----+-----+-------------------+
|2014|   12| -1.838561940815072|
|2014|   11| 2.9202591036414574|
|2014|   10| 2.6506731357636166|
|2014|    9|-0.9561130563336455|
|2014|    8|0.34340513857551613|
|2014|    7| 0.7707767430662216|
|2014|    6| 0.5867637800883099|
|2014|    5| 1.1918083405986657|
|2014|    4| 1.8507542360173952|
|2014|    3| 3.8866128652955414|
|2014|    2| 3.4851983771565043|
|2014|    1| 3.1917645844887845|
|2013|   12|  4.672728381765573|
|2013|   11| 0.3435924369747902|
|1990|    7| 2.6110993237113824|
|1990|    2|   62.8101983771565|
|1970|    7| 16.111099323711382|
|1970|    2| 22.810198377156503|
|1961|    5| 1.0272486195873274|
|1961|    4|  1.581865347128507|
+----+-----+-------------------+
only showing top 20 rows



In [10]:
a = sc.parallelize([1,2,3, 4])
b = sc.parallelize([1,2,3,3])
a.dot(b)

AttributeError: 'RDD' object has no attribute 'dot'

In [52]:
def haversine(lon1, lat1): 
    return lon1+lat1

sc.stop()
sc = SparkContext(appName="exercise")
stations = sc.textFile("stations.csv")
lines_stations = stations.map(lambda line:line.split(";"))
stations = lines_stations.map(lambda x:(x[0],haversine(float(x[3]),float(x[4]))))
print(*stations.take(10),sep="\n")
m=sc.parallelize(stations.collect()).collectAsMap()
stations=sc.broadcast(m)


('102170', 73.1326)
('102190', 73.0056)
('102200', 73.0103)
('102210', 73.0039)
('102390', 73.52912)
('102540', 73.4909)
('103050', 73.6166)
('103080', 73.0983)
('103090', 73.94890000000001)
('103100', 73.9557)


In [61]:
stations.value.values()

dict_values([73.1326, 73.0056, 73.0103, 73.0039, 73.52912, 73.4909, 73.6166, 73.0983, 73.94890000000001, 73.9557, 74.3775, 74.38730000000001, 74.8661, 74.5377, 74.60000000000001, 75.4659, 75.0787, 75.2632, 75.46770000000001, 75.24557, 75.5333, 75.5226, 75.60679999999999, 75.9373, 75.9213, 75.96260000000001, 75.61919999999999, 76.2793, 76.0267, 75.7423, 76.0375, 76.0666, 75.95, 76.4053, 76.40350000000001, 76.3166, 76.4815, 77.2458, 76.7213, 77.3999, 76.8902, 77.3931, 77.4276, 77.13329999999999, 78.1075, 78.1429, 78.14410000000001, 77.7333, 77.78020000000001, 78.2898, 77.82300000000001, 77.8807, 78.2916, 78.1121, 78.81, 78.9031, 78.4022, 79.1988, 79.3666, 78.9028, 78.7666, 73.9495, 74.1365, 74.1215, 74.17150000000001, 74.74340000000001, 74.27212, 74.4284, 74.193, 74.8501, 74.8777, 74.28320000000001, 75.5899, 75.29249999999999, 75.29299999999999, 76.319, 75.75, 75.7869, 76.4423, 76.652, 76.6554, 75.9468, 76.35, 76.7666, 76.8363, 77.0788, 77.21360000000001, 77.8177, 77.8095, 77.87780000000